In [163]:
import keras

from keras.datasets import boston_housing
from keras import layers
from keras.optimizers import RMSprop

from sklearn.model_selection import KFold

In [164]:
epochs = 20
folds = 10

In [165]:
(train_x, train_y), (test_x, test_y) = boston_housing.load_data()

print(f'Training input shape : {train_x.shape}')
print(f'Training output shape : {train_y.shape}\n')

print(f'Test input shape : {test_x.shape}')
print(f'Test output shape : {test_y.shape}')

Training input shape : (404, 13)
Training output shape : (404,)

Test input shape : (102, 13)
Test output shape : (102,)


In [166]:
#Нормализуем данные по первой сигме для каждого входного параметра.
mean = train_x.mean(axis=0)
train_x -= mean
std = train_x.std(axis=0)
train_x /= std

#Тестовые данные обязательно нормализовать по распределнию от обучающих данных.
test_x -= mean
test_x /= std

In [167]:
# Модель
x = keras.Input(shape=(13,))
y = layers.Dense(256, activation='relu')(x)
y = layers.Dense(1)(y)

model = keras.Model(inputs=x, outputs=y)
model.summary()

#model.compile(optimizer=RMSprop(),loss='mse',metrics=['mae'])
model.compile(optimizer='rmsprop',loss='mse',metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_22 (InputLayer)        (None, 13)                0         
_________________________________________________________________
dense_61 (Dense)             (None, 256)               3584      
_________________________________________________________________
dense_62 (Dense)             (None, 1)                 257       
Total params: 3,841
Trainable params: 3,841
Non-trainable params: 0
_________________________________________________________________


In [168]:
#Тренируем по фолдам
for i, (train_index, test_index) in enumerate(KFold(folds).split(train_x, train_y)):
    print(f'### Fold: {i}, Epochs: {epochs} ###########################################')
    model.fit(
        train_x[train_index],
        train_y[train_index],
        batch_size=1,
        epochs=epochs,
        verbose=0,
    )
    loss, acc = model.evaluate(train_x[test_index], train_y[test_index], verbose=0)
    print(f'val_loss: {loss}, val_acc: {acc}\n')


### Fold: 0, Epochs: 20 ###########################################
val_loss: 16.583508619448033, val_acc: 0.024390244084160503

### Fold: 1, Epochs: 20 ###########################################
val_loss: 2.9390544135396075, val_acc: 0.04878048780487805

### Fold: 2, Epochs: 20 ###########################################
val_loss: 8.443187783404094, val_acc: 0.024390243902439025

### Fold: 3, Epochs: 20 ###########################################
val_loss: 10.48260470134456, val_acc: 0.0

### Fold: 4, Epochs: 20 ###########################################
val_loss: 5.671169662475586, val_acc: 0.025

### Fold: 5, Epochs: 20 ###########################################
val_loss: 5.370118188858032, val_acc: 0.025

### Fold: 6, Epochs: 20 ###########################################
val_loss: 4.76997938156128, val_acc: 0.025

### Fold: 7, Epochs: 20 ###########################################
val_loss: 19.232290267944336, val_acc: 0.0

### Fold: 8, Epochs: 20 ##############################

In [169]:
#Валидируем на тестовых данных, которые не участвовали в тренировке.
score = model.evaluate(test_x, test_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 11.950872757855583
Test accuracy: 0.0392156862745098
